In [34]:
import re
import mysql.connector
import numpy as np
import xml.etree.ElementTree as ET

In [54]:
mydb = mysql.connector.connect(
    host='127.0.0.1', 
    user='Yuxin Xiao', 
    password='19961002XYX',
    database='CS510Project')
mycursor = mydb.cursor()

In [ ]:
mycursor.execute("ALTER TABLE Sentences ADD COLUMN Id INT AUTO_INCREMENT PRIMARY KEY FIRST")

In [ ]:
mycursor.execute("CREATE TABLE Sentences (QA_id INT, Subject_id INT, Sentence_id INT, Sentences TEXT)")
mycursor.execute("CREATE TABLE Tags (QA_id INT, Category VARCHAR(255), Main_category VARCHAR(255), Subcategory VARCHAR(255))")
mycursor.execute("CREATE TABLE Words (Word VARCHAR(255), Count INT, Inverted_index LONGTEXT)")

In [ ]:
tree = ET.parse('data/manner.xml')
root = tree.getroot()

In [ ]:
regex = r"(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s"
subst = "\\n"

In [ ]:
def insert_sentences(QA_id, QA_pair):
    sql = ('INSERT INTO Sentences '
           '(QA_id, Subject_id, Sentence_id, Sentences) '
           'VALUES (%s, %s, %s, %s)')
    
    question = QA_pair[0].findall('subject')[0].text
    question = re.sub(regex, subst, question, 0, re.MULTILINE).split('\n')
    question = [sentence.lstrip().replace('<br />', '') for sentence in question]
    question = list(filter(None, question))
    for sentence_id, sentence in enumerate(question):
        val = (QA_id+1, 0, sentence_id+1, sentence)
        mycursor.execute(sql, val)
    
    
    answers = QA_pair[0].findall('nbestanswers')[0]
    for answer_id, answer in enumerate(answers):
        answer = re.sub(regex, subst, answer.text, 0, re.MULTILINE).split('\n')
        answer = [sentence.lstrip().replace('<br />', '') for sentence in answer]
        answer = list(filter(None, answer))
        
        for sentence_id, sentence in enumerate(answer):
            val = (QA_id+1, answer_id+1, sentence_id+1, sentence)
            mycursor.execute(sql, val)

In [ ]:
def insert_tag(QA_id, QA_pair):
    def extract_tags(category_name):
        tags = QA_pair[0].findall(category_name)
        if not tags:
            return None
        tags = tags[0].text
        tags = re.sub(regex, subst, tags, 0, re.MULTILINE).split('&')
        tags = [tag.lstrip().rstrip() for tag in tags]
        tags = ', '.join(tags)
        return tags
    cat = extract_tags('cat')
    maincat = extract_tags('maincat')
    subcat = extract_tags('subcat')
    sql = ('INSERT INTO Tags '
           '(QA_id, Category, Main_category, Subcategory) '
           'VALUES (%s, %s, %s, %s)')
    val = (QA_id+1, cat, maincat, subcat)
    mycursor.execute(sql, val)

In [ ]:
for QA_id, QA_pair in enumerate(root):
    insert_sentences(QA_id, QA_pair)
    insert_tag(QA_id, QA_pair)
    if QA_id % 5000 == 0:
        print(QA_id)

In [ ]:
mydb.commit()

In [ ]:
mycursor.execute("SELECT Sentences FROM Sentences")
myresult = mycursor.fetchall()
with open('All Sentences.txt','w') as file:
    for sentence in myresult:
        sentence = str(sentence).encode('utf-8','ignored')
        file.write("%s\n" % sentence)

In [3]:
mycursor.execute("SELECT * FROM Sentences")
myresult = mycursor.fetchall()

In [43]:
mycursor.execute("DROP TABLE Words")
mycursor.execute("CREATE TABLE Words (Word TEXT, Count INT, Inverted_index LONGTEXT)")

In [5]:
myresult = myresult[:100000]
len(myresult)

100000

In [6]:
myresult[0]

(1, 1, 0, 1, 'How to handle a 1.5 year old when hitting?')

In [7]:
print("Creating Dictionary")
print(len(myresult))
   
count_dict = {}
index_dict = {}

num = 0
for row in myresult:
    
    num += 1
    if num % 20000 == 0:
        print(num)
    
    index = str(row[0])
    sentence = re.sub(r'[^\w\s]', "", row[4]).split()
        
    for word in sentence:
        if word.isdigit():
            continue
        word = word.lower()
        
        if word in count_dict:
            count_dict[word] += 1
            index_dict[word] += (' ' + index)
        else:
            count_dict[word] = 1
            index_dict[word] = index

Creating Dictionary
100000
20000
40000
60000
80000
100000


In [33]:
count_keys, count_values = list(count_dict.keys()), list(count_dict.values())
index_keys, index_values = list(index_dict.keys()), list(index_dict.values())

In [35]:
index_len = [len(index_value) for index_value in index_values]

In [47]:
key_len = [len(index_key) for index_key in index_keys]

In [44]:
print("Writing into Database")
print(len(count_dict.keys()))

num = 0
for word in count_dict.keys():
    
    num += 1
    if num % 5000 == 0:
        print(num)
    
    count = count_dict[word]
    index = index_dict[word]

    insert_sql = "INSERT INTO Words (Word, Count, Inverted_index) VALUES (%s, %s, %s)"
    insert_val = (word, count, index)
    mycursor.execute(insert_sql, insert_val)

Writing into Database
48248
5000
10000
15000
20000
25000
30000
35000
40000
45000


In [ ]:
batch_size = 50000
sentences_count = len(myresult)
epoch_count = int(sentences_count / batch_size + 1)
print(epoch_count)

for epoch in range(epoch_count):
    
    print(epoch)
    
    count_dict = {}
    index_dict = {}
    
    for row in myresult[:(epoch+1)*batch_size]:
        QA_id, subject_id, sentence_id = row[0], row[1], row[2]
        sentence = re.sub(r'[^\w\s]', "", row[3]).split()
        index = "%d-%d-%d" % (QA_id, subject_id, sentence_id)
        
        for word in sentence:
            if word.isdigit():
                continue
            word = word.lower()
            
            if word in count_dict:
                count_dict[word] += 1
                index_dict[word] += (', ' + index)
            else:
                count_dict[word] = 1
                index_dict[word] = index
        
    for word in count_dict.keys():
        count = count_dict[word]
        index = index_dict[word]
        
        select_sql = "SELECT * FROM Words WHERE Word = %s"        
        mycursor.execute(select_sql, (word,))
        corr_row = mycursor.fetchall()
        
        if not corr_row:
            insert_sql = "INSERT INTO Words (Word, Count, Inverted_index) VALUES (%s, %s, %s)"
            insert_val = (word, count, index)
            mycursor.execute(insert_sql, insert_val)
        else:
            update_count = corr_row[0][1] + count
            update_index = corr_row[0][2] + ", " + index
            update_sql = "UPDATE Words SET Count = %s, Inverted_index = %s WHERE Word = %s"
            update_val = (update_count, update_index, word)
            mycursor.execute(update_sql, update_val)

In [57]:
mycursor.execute("SELECT * FROM Words")
test = mycursor.fetchall()

In [60]:
test[100]

('warn', 3, '11 78200 94784')

In [45]:
mydb.commit()

In [46]:
mycursor.close()
mydb.close()

In [ ]:
mycursor.execute("DROP TABLE Sentences")
mycursor.execute("DROP TABLE Tags")
mycursor.execute("DROP TABLE Words")

In [ ]:
nonletters = '^[^a-zA-z]*|[^a-zA-Z]*$'
with open('All Sentences.txt', 'r') as file:
    sentences = file.read().split('\nb')
    sentences[0] = sentences[0][2:]
    sentences = [re.sub(nonletters, '', sentence[:-2].replace('\\','')) for sentence in sentences]